In [1]:
import utils.paths as pth
from utils.dataset import *
from utils.plot_utils import *

from pkg_resources import get_platform

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output


gps_coords = 46.08247, 7.29364
peaks_df, observer_height = pth.get_mtn_geometry(gps_coords)

Opening tile number  0  of  49
Opening tile number  1  of  49
Opening tile number  2  of  49
Opening tile number  3  of  49
Opening tile number  4  of  49
Opening tile number  5  of  49
Opening tile number  6  of  49
Opening tile number  7  of  49
Opening tile number  8  of  49
Opening tile number  9  of  49
Opening tile number  10  of  49
Opening tile number  11  of  49
Opening tile number  12  of  49
Opening tile number  13  of  49
Opening tile number  14  of  49
Opening tile number  15  of  49
Opening tile number  16  of  49
Opening tile number  17  of  49
Opening tile number  18  of  49
Opening tile number  19  of  49
Opening tile number  20  of  49
Opening tile number  21  of  49
Opening tile number  22  of  49
Opening tile number  23  of  49
Opening tile number  24  of  49
Opening tile number  25  of  49
Opening tile number  26  of  49
Opening tile number  27  of  49
Opening tile number  28  of  49
Opening tile number  29  of  49
Opening tile number  30  of  49
Opening tile numbe

In [42]:
import importlib
importlib.reload(pth)

mdf_list, tdf_list, amdf_list  = [], [], []
for month in np.arange(1,13):
    # td = datetime.timedelta(days = interval)
    # final_date = start_date + n_intervals * td
    year = 2021
    day = 1
    date = {'year':year,'month':month,'day':day}
    date = datetime.date(year = date['year'], month = date['month'], day = date['day'])
    mdf_,tdf_,amdf_  = pth.get_data(gps_coords, observer_height, peaks_df, date)
    mdf_list.append(mdf_)
    tdf_list.append(tdf_)
    amdf_list.append(amdf_)


month = 6

mdf = mdf_list[month]
tdf = tdf_list[month]

peak_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.peak_angle,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'black'
                # fill = 'tozeroy'
                )

diff_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.el_diff,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'gold'
                # fill = 'tozeroy'
                )

sun_line = dict(
                type = 'scatter',
                x = mdf.loc[ mdf.daylight == 'day' ].bearing,
                y = mdf.loc[ mdf.daylight == 'day' ].elevation,
                line = dict(
                    color = 'white',
                    width = 0.5),
            )

pio.templates.default = "simple_white"

fig = go.Figure()

fig.add_traces([
    peak_lines,
    diff_lines,
    sun_line
])

annotations = []
gdf = tdf.loc[tdf.elevation > 0].groupby('time')
for hour, df in gdf:
    df = df.sort_values('date').reset_index()
    # print(df.head)
    x = float( df['azimuth' ] )
    y = float( df['elevation' ] )
    text = hour
    grad = float( df.grad )
    fig.add_trace(
        go.Scatter(
            x=[ x, x + 2 * grad],
            y=[ y, y - 2] ,
            line = dict( color="white", width=1), 
            # fill = 'toself',
            marker = None,
            mode = 'lines',
            showlegend = False
        )
    )
    # markers.append(make_marker_dict(x,y,hour))
    annotations.append( dict(
        text = str(text) + ':00',
        x = x,
        y = y,
        xanchor = 'center',
        yanchor = 'middle',
        xshift = 10 * grad,
        yshift = -20,
        showarrow = False,
        font = dict(color = 'white'),
        opacity = 1
    )    )

fig.update_layout( 
    annotations = annotations,
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 22.5, 45.0, 67.5, 90.0, 112.5, 135.0, 157.5, 180.0, 202.5, 225.0, 247.5, 270.0, 292.5, 315, 337.5] ,
        ticktext = ['N', 'ENE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NWN'] 
    ),
    yaxis = dict(
        range = [0, 80],
        anchor = 'free',
        position = 0.5,
        visible = False
    ),
    showlegend=False,
    autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
)

In [30]:
tdf

,date,time,azimuth,elevation,grad,midday_elev
0,2021-07-01,4,46.309709,-6.667967,0.735840,66.987801
1,2021-07-01,5,57.530983,1.589095,0.818977,66.987801
2,2021-07-01,6,67.575254,10.562584,0.943874,66.987801
3,2021-07-01,7,77.634373,20.565652,1.001980,66.987801
4,2021-07-01,8,87.931752,30.96304,0.965064,66.987801
5,2021-07-01,9,99.355614,41.421948,0.826141,66.987801
6,2021-07-01,10,112.837177,51.137412,0.614301,66.987801
7,2021-07-01,11,132.002597,60.013244,0.360884,66.987801
8,2021-07-01,12,160.098198,65.941832,0.122738,66.987801
9,2021-07-01,13,194.982598,66.399891,-0.090365,66.987801
